## CNN 

In [3]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
torch.set_printoptions(linewidth=120)

In [34]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 12, 5)
        
        self.fc1 = nn.Linear(12 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 10)
    
    def forward(self, t):
        # hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # output layer, softmax bei crossentropy
        t = self.fc3(t)
 
        return t

In [19]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [38]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

#net = Net()
optimizer = optim.Adam(net.parameters(), lr=0.01)
train_loader = torch.utils.data.DataLoader(
     train_set
    ,batch_size=100
    ,shuffle=True
)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = net(images)    # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward()        # Calculate Gradients
        optimizer.step()       # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "accuracy:", round(total_correct/len(train_set),3),
        "loss:", total_loss
    )

epoch 0 total_correct: 54054 accuracy: 0.901 loss: 170.9793567210436
epoch 1 total_correct: 54263 accuracy: 0.904 loss: 159.0782235264778
epoch 2 total_correct: 54105 accuracy: 0.902 loss: 167.92552763223648
epoch 3 total_correct: 53878 accuracy: 0.898 loss: 174.5344119593501
epoch 4 total_correct: 54337 accuracy: 0.906 loss: 158.0523509606719
epoch 5 total_correct: 54361 accuracy: 0.906 loss: 156.78790247440338
epoch 6 total_correct: 54275 accuracy: 0.905 loss: 158.47815699875355
epoch 7 total_correct: 53717 accuracy: 0.895 loss: 178.11085463315248
epoch 8 total_correct: 54323 accuracy: 0.905 loss: 155.68321303278208
epoch 9 total_correct: 54509 accuracy: 0.908 loss: 151.74283998459578


#### Anwenden auf Testdaten

In [33]:
test_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=False
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

test_loader = torch.utils.data.DataLoader(test_set
    ,batch_size=100
)

correct = 0
for batch in test_loader:
    images, labels = batch 
    preds = net(images)
    correct += get_num_correct(preds,labels)
    
print("correct:",correct/len(test_set))

correct: 0.8634


In [29]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(28 * 28, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 10)
    
    def forward(self, t):

        t = t.reshape(-1, 28 * 28)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)
        
        # softmax bei crossentropy
        t = self.fc3(t)
        return t

Nach 10 Epochen ohne conv-Layer: ca 89% Train, 86% Test